<a href="https://colab.research.google.com/github/nceder/qpb4e/blob/main/code/Chapter%2024/Chapter_24.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 24 Saving Data

# 24.2 SQLite: Using the sqlite3 database

In [1]:
import sqlite3
conn = sqlite3.connect("datafile.db")

In [2]:
cursor = conn.cursor()
cursor

In [3]:
cursor.execute("create table people (id integer primary key, name text, count integer)")
cursor.execute("insert into people (name, count) values ('Bob', 1)")
cursor.execute("insert into people (name, count) values (?, ?)",
               ("Jill", 15))
conn.commit()

In [4]:
cursor.execute("insert into people (name, count) values (:username, :usercount)",
                  {"username": "Joe", "usercount": 10})

In [5]:
result = cursor.execute("select * from people")
print(result.fetchall())

[(1, 'Bob', 1), (2, 'Jill', 15), (3, 'Joe', 10)]


In [6]:
result = cursor.execute("select * from people where name like :name",
                        {"name": "bob"})
print(result.fetchall())

[(1, 'Bob', 1)]


In [7]:
cursor.execute("update people set count=? where name=?", (20, "Jill"))
result = cursor.execute("select * from people")
print(result.fetchall())

[(1, 'Bob', 1), (2, 'Jill', 20), (3, 'Joe', 10)]


In [8]:
result = cursor.execute("select * from people")
for row in result:
    print(row)


(1, 'Bob', 1)
(2, 'Jill', 20)
(3, 'Joe', 10)


In [9]:
cursor.execute("update people set count=? where name=?", (20, "Jill"))
conn.commit()
conn.close()

# 24.4 Making database handling easier with an ORM

## 24.4.1 SQLAlchemy

In [10]:
from sqlalchemy import create_engine, select, MetaData, Table, Column, Integer, String
from sqlalchemy.orm import sessionmaker

In [11]:
dbPath = 'datafile2.db'
engine = create_engine('sqlite:///%s' % dbPath)
metadata = MetaData()
people  = Table('people', metadata,
                Column('id', Integer, primary_key=True),
                Column('name', String),
                Column('count', Integer),
               )
Session = sessionmaker(bind=engine)
session = Session()
metadata.create_all(engine)

In [12]:
people_ins = people.insert().values(name='Bob', count=1)
str(people_ins)

'INSERT INTO people (name, count) VALUES (:name, :count)'

In [13]:
session.execute(people_ins)

In [14]:
session.commit()

In [15]:
session.execute(people_ins, [
    {'name': 'Jill', 'count':15},
    {'name': 'Joe', 'count':10}
])

In [16]:
session.commit()
people_query = select(people)
result = session.execute(people_query)
for row in result:
    print(row)


(1, 'Bob', 1)
(2, 'Jill', 15)
(3, 'Joe', 10)


In [17]:
result = session.execute(select(people).where(people.c.name == 'Jill'))
for row in result:
    print(row)


(2, 'Jill', 15)


In [18]:
result = session.execute(people.update().values(count=20).where (people.c.name == 'Jill'))
session.commit()
result = session.execute(select(people).where(people.c.name == 'Jill'))
for row in result:
    print(row)


(2, 'Jill', 20)


### Mapping table objects to classes

In [19]:
from sqlalchemy.orm import declarative_base
Base = declarative_base()
class People(Base):
    __tablename__ = "people"
    id = Column(Integer, primary_key=True)
    name = Column(String)
    count = Column(Integer)

results = session.query(People).filter_by(name='Jill')
for person in results:
    print(person.id, person.name, person.count)


2 Jill 20


In [20]:
new_person = People(name='Jane', count=5)
session.add(new_person)
session.commit()
results = session.query(People).all()
for person in results:
    print(person.id, person.name, person.count)


1 Bob 1
2 Jill 20
3 Joe 10
4 Jane 5


In [21]:
jill = session.query(People).filter_by(name='Jill').first()
jill.name

'Jill'

In [22]:
jill.count = 22
session.add(jill)
session.commit()
results = session.query(People).all()
for person in results:
    print(person.id, person.name, person.count)


1 Bob 1
2 Jill 22
3 Joe 10
4 Jane 5


In [23]:
jane = session.query(People).filter_by(name='Jane').first()
session.delete(jane)
session.commit()
jane = session.query(People).filter_by(name='Jane').first()
print(jane)

None


## 24.4.2 Using Alembic for database schema changes

In [24]:
! rm -rf alembic/

In [25]:
! pip install alembic
! alembic init alembic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.2/233.2 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 5.9 MB/s eta 0:00:00
Creating directory '/content/alembic' ...  done
Creating directory '/content/alembic/versions' ...  done
Generating /content/alembic/env.py ...  done
Generating /content/alembic/script.py.mako ...  done
Generating /content/alembic/README ...  done
Generating /content/alembic.ini ...  done
Please edit configuration/connection/logging settings in '/content/alembic.ini' before proceeding.


In [26]:
# This cell will update the alembic.ini file

! sed -i 's/driver:\/\/user:pass@localhost\/dbname/sqlite:\/\/\/datafile.db/' alembic.ini

In [27]:
# This cell creates the first revision script
result = ! alembic revision -m "create an address table"
filename= result[0].replace('Generating ', "").replace(" ...  done","")
version = filename.split("/")[-1].split("_")[0]

In [28]:
# This cell updates the revision script's upgrade() and downgrade() functions
upgrade_cmd = """def upgrade() -> None:
    op.create_table(
        'address',
        sa.Column('id', sa.Integer, primary_key=True),
        sa.Column('address', sa.String(50), nullable=False),
        sa.Column('city', sa.String(50), nullable=False),
        sa.Column('state', sa.String(20), nullable=False),
    )
"""
downgrade_cmd = """def downgrade() -> None:
    op.drop_table('address')"""

version_file = open(filename).read()
version_file = version_file.replace("""def upgrade() -> None:
    pass""", upgrade_cmd)
version_file = version_file.replace("""def downgrade() -> None:
    pass""", downgrade_cmd)
print(version_file)

open(filename, "w").write(version_file)

"""create an address table

Revision ID: def74d4e1932
Revises: 
Create Date: 2024-09-27 02:02:49.374125

"""
from typing import Sequence, Union

from alembic import op
import sqlalchemy as sa


# revision identifiers, used by Alembic.
revision: str = 'def74d4e1932'
down_revision: Union[str, None] = None
branch_labels: Union[str, Sequence[str], None] = None
depends_on: Union[str, Sequence[str], None] = None


def upgrade() -> None:
    op.create_table(
        'address',
        sa.Column('id', sa.Integer, primary_key=True),
        sa.Column('address', sa.String(50), nullable=False),
        sa.Column('city', sa.String(50), nullable=False),
        sa.Column('state', sa.String(20), nullable=False),
    )



def downgrade() -> None:
    op.drop_table('address')



771

In [29]:
for table in metadata.sorted_tables:
    print(table.name)

people


In [31]:
! alembic upgrade head

INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.


In [32]:
metadata.sorted_tables

[Table('people', MetaData(), Column('id', Integer(), table=<people>, primary_key=True, nullable=False), Column('name', String(), table=<people>), Column('count', Integer(), table=<people>), schema=None)]

In [33]:
Session = sessionmaker(bind=engine)
session = Session()


In [34]:
metadata.reflect(engine)
metadata.tables.keys()
session.commit()

In [35]:
! alembic downgrade -1


INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Running downgrade def74d4e1932 -> , create an address table


In [37]:
for table in metadata.sorted_tables:
    print(table.name)

people


# 24.6 key:value stores with Redis

In [38]:
!pip install redis

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.6/255.6 kB 4.5 MB/s eta 0:00:00


### NOTE: You must create an account at redis.io or on another redis server for the examples to work properly.

Be sure to use the correct hostname and password for your account in cell below.

In [39]:
import redis

r = redis.Redis(
    # set your host here
    host='',
    port=10032,
    # set your password below
    password='')

#### Basic operations

In [40]:
r.keys()

[]

#### Array operations

In [41]:
r.set('a_key', 'my value')

True

In [42]:
r.keys()

[b'a_key']

In [43]:
v = r.get('a_key')
v

b'my value'

In [44]:
r.incr('counter')

1

In [45]:
r.get('counter')

b'1'

In [46]:
r.incr('counter')

2

In [47]:
r.get('counter')

b'2'

In [48]:
r.rpush("words", "one")

1

In [49]:
r.rpush("words", "two")

2

In [50]:
r.lrange("words", 0, -1)

[b'one', b'two']

In [51]:
r.rpush("words", "three")

3

In [52]:
r.lrange("words", 0, -1)

[b'one', b'two', b'three']

In [53]:
r.llen("words")

3

In [54]:
r.lpush("words", "zero")

4

In [55]:
r.lrange("words", 0, -1)

[b'zero', b'one', b'two', b'three']

In [56]:
r.lrange("words", 2, 2)

[b'two']

In [57]:
r.lindex("words", 1)

b'one'

In [58]:
r.lindex("words", 2)

b'two'

### Expiration of values

In [59]:
r.setex("timed", 10,  "10 seconds")

True

In [60]:
r.pttl("timed")

8647

In [63]:
r.pttl("timed")

-2

In [64]:
b"timed" in r.keys()

False

# MongoDB Atlas

In [65]:
! pip install pymongo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 18.1 MB/s eta 0:00:00


### NOTE: you must have an account at mongondb.net or on another Mongo server for this code to work properly.

You may also have to authorize Access from Anywhere to connect to your DB.

In [70]:
from pymongo import MongoClient
client = MongoClient(host='*** connection string here ***')   #A

In [71]:
import datetime
a_document = {'name': 'Jane',
              'age': 34,
              'interests': ['Python', 'databases', 'statistics'],
              'date_added': datetime.datetime.now()
}
db = client.my_data     #A
collection = db.docs   #B
result = collection.find_one()  #C
db.list_collection_names()

[]

In [72]:
collection.insert_one(a_document)


InsertOneResult(ObjectId('66f616d8b15e1d493ca8b8d0'), acknowledged=True)

In [73]:
collection.find_one()    #A

{'_id': ObjectId('66f616d8b15e1d493ca8b8d0'),
 'name': 'Jane',
 'age': 34,
 'interests': ['Python', 'databases', 'statistics'],
 'date_added': datetime.datetime(2024, 9, 27, 2, 21, 7, 521000)}

In [82]:
from bson.objectid import ObjectId

# use ObjectId from above in code below
collection.find_one({"_id":ObjectId('66f616d8b15e1d493ca8b8d0')})  #B


In [76]:
# use ObjectId from above in code below
collection.update_one({"_id":ObjectId('66f616d8b15e1d493ca8b8d0')}, {"$set": {"name":"Ann"}})       #C

UpdateResult({'n': 1, 'electionId': ObjectId('7fffffff00000000000002ba'), 'opTime': {'ts': Timestamp(1727403761, 6), 't': 698}, 'nModified': 1, 'ok': 1.0, '$clusterTime': {'clusterTime': Timestamp(1727403761, 6), 'signature': {'hash': b"\xdc\t\xa3\x9d\x0291r+\x05h\xd1S^'\xa4L%)\xe8", 'keyId': 7368510527980437523}}, 'operationTime': Timestamp(1727403761, 6), 'updatedExisting': True}, acknowledged=True)

In [77]:
# use ObjectId from above in code below
collection.find_one({"_id":ObjectId('66f616d8b15e1d493ca8b8d0')})

{'_id': ObjectId('66f616d8b15e1d493ca8b8d0'),
 'name': 'Ann',
 'age': 34,
 'interests': ['Python', 'databases', 'statistics'],
 'date_added': datetime.datetime(2024, 9, 27, 2, 21, 7, 521000)}

In [78]:
# use ObjectId from above in code below
collection.replace_one({"_id":ObjectId('66f616d8b15e1d493ca8b8d0')}, {"name":"Maria"})                 #D

UpdateResult({'n': 1, 'electionId': ObjectId('7fffffff00000000000002ba'), 'opTime': {'ts': Timestamp(1727403772, 14), 't': 698}, 'nModified': 1, 'ok': 1.0, '$clusterTime': {'clusterTime': Timestamp(1727403772, 14), 'signature': {'hash': b'\x95\x93\xd2\xe2<[\xe9W\xb4\x053\xc1Wf\x01\xc7b\xe5\xc4a', 'keyId': 7368510527980437523}}, 'operationTime': Timestamp(1727403772, 14), 'updatedExisting': True}, acknowledged=True)

In [79]:
# use ObjectId from above in code below
collection.find_one({"_id":ObjectId('66f616d8b15e1d493ca8b8d0')})

{'_id': ObjectId('66f616d8b15e1d493ca8b8d0'), 'name': 'Maria'}

In [80]:
# use ObjectId from above in code below
collection.delete_one({"_id":ObjectId('66f616d8b15e1d493ca8b8d0')}) #E

DeleteResult({'n': 1, 'electionId': ObjectId('7fffffff00000000000002ba'), 'opTime': {'ts': Timestamp(1727403777, 14), 't': 698}, 'ok': 1.0, '$clusterTime': {'clusterTime': Timestamp(1727403777, 14), 'signature': {'hash': b'O\x08\xad\xff\xc3[E=\xc8\x06\xd0\xc4\xe5^\x0b\xb8\x1f\x04\x04\xd7', 'keyId': 7368510527980437523}}, 'operationTime': Timestamp(1727403777, 14)}, acknowledged=True)

In [ ]:
collection.find_one()

In [ ]:
db.list_collection_names()

['docs']

In [ ]:
collection.drop()
db.list_collection_names()

[]